In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

**Torch, Numpy stuff**

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device.type)

# ---

torch.manual_seed(42)
np.random.seed(42)

cuda


---
**Training Data**

Demonstration data contains a list of **d** scenes:

- Each scene contains a list of trajectories of **p** people, where p is **not constant**
- A trajectory is a list of **t** states, where t = 400
- A state is a **s** = 4 dimensional variable
 - State = (d<sub>goal<sub>x</sub></sub>, d<sub>goal<sub>y</sub></sub>, v<sub>x</sub>, v<sub>y</sub>)
    
> Shape of data is (d, p, t, s) => (d, p, 400, 4)

---

Train, test, val split: 0.8, 0.01, 0.19

In [ ]:
from sklearn.model_selection import train_test_split

def data_from_demonstrations(obs_dims=[0,1], tar_dims=[2,3], path="../data/input/"):
    data = np.load(f"{path}/states_processed.npy")
    X, Y = data[:,:,:,obs_dims], data[:,:,:,tar_dims]
    return X, Y


# for testing purposes
def synthetic_data():
    X = np.random.uniform(0, 1, 400)
    Y = np.sin(X*2*np.pi)
    return X, Y

#X, Y = data_from_demonstrations()
X, Y = synthetic_data()

X_train, X_rem, Y_train, Y_rem = train_test_split(X, Y, train_size=0.8)

# 0.2 * 0.05 = 0.01 (test size in entire data)
test_sz = 0.05
X_val, X_test, Y_val, Y_test = train_test_split(X_rem, Y_rem, test_size=test_sz)



def sample_training_batch():
    pass